<p style="float: left;"><a href="merge-sort.ipynb" target="_blank">Previous</a></p>
<p style="float: right;"><a href="for-comprehensions.ipynb" target="_blank">Next</a></p>
<p style="text-align:center;">Tour of Scala</p>
<div style="clear: both;"></div>

# Definition of class List II: Higher-Order Methods

**Functional programming** languages **enable programmers to write general functions
which implement patterns by means of higher order functions**. We now
discuss a set of commonly used higher-order functions, which are implemented as
methods in class `List`.

We can identify several patterns of computation over lists, like:

* transforming every element of a list in some way.
* extracting from a list all elements satisfying a criterion.
* combine the elements of a list using some operator.

## Mapping

A common operation is to transform each element of a list
and then return the lists of results.

```scala
def map[B](f: A => B): List[B] = this match {
    case Nil     => this
    case x :: xs => f(x) :: xs.map(f)
}
```


For instance, to scale each element of a list by a
given factor:

In [1]:
import scala.util.Random

val rand    = new Random
val numbers =  for (x <- 1 to 3) yield rand.nextDouble()
val randInt = rand.nextInt()
val scale   = (if (randInt >= 0) randInt else -1 * randInt) % 5 + 1

def scaleList(xs: List[Double], factor: Double): List[Double] = xs match {
    case Nil      => Nil
    case x :: xs1 => x * factor :: scaleList(xs1, factor)
}

val scaledNumbers = scaleList(numbers.toList, scale)

import scala.util.Random
rand: Random = scala.util.Random@4bde16fa
numbers: IndexedSeq[Double] = Vector(
  0.18482415684938847,
  0.5007184003872804,
  0.506468992252953
)
randInt: Int = -1694641057
scale: Int = 3
defined function scaleList
scaledNumbers: List[Double] = List(
  0.5544724705481654,
  1.5021552011618413,
  1.5194069767588592
)

As another example, consider the problem of returning a given column of a matrix
which is represented as a list of rows, where each row is again a list.

In [2]:
import scala.util.Random

def column[A](xs: List[List[A]], index: Int): List[A] =
    xs map (row => row(index))

val (rows, columns) = (2, 3)
val grid = (
    for (x <- 1 to rows) 
        yield (for (x <- 1 to columns) yield rand.nextDouble()).toList
).toList

val lastCol = column(grid, columns - 1)

import scala.util.Random
defined function column
rows: Int = 2
columns: Int = 3
grid: List[List[Double]] = List(
  List(0.12036256996378802, 0.6248197415837481, 0.638229595811933),
  List(0.5750946850293309, 0.44025428502316444, 0.5130119776095965)
)
lastCol: List[Double] = List(0.638229595811933, 0.5130119776095965)

## Foreach

Closely related to `map` is the `foreach` method, which applies a given function to all
elements of a list, but does not construct a list of results. The function is thus applied
only for its side effect.

```scala
def foreach(f: A => Unit) {
    this match {
        case Nil     => ()
        case x :: xs => f(x); xs.foreach(f)
    }
}
```


For instance, printing numbers from 1 to 3.

In [3]:
(1 to 3).toList foreach println

1
2
3


## Filtering

Another common operation selects from a list all elements fulfilling a given criterion.

```scala
def filter(p: A => Boolean): List[A] = this match {
    case Nil     => this
    case x :: xs => if (p(x)) x :: xs.filter(p) else xs.filter(p)
}
```

For instance, to return a list of all positive elements in some
given lists of integers:

```scala
import scala.util.Random

val rand = new Random
val randNumbers = for (_ <- 1 to 5) yield rand.nextInt()
val positiveNumbers = randNumbers filter (x => x > 0)

```

## Folding and reducing

Another common operation is to combine the elements of a list with some operator. For instance:

* `sum(List(x1, ..., xn))`  $ = 0 + x_1 + \dots + x_n$
* `product(List(x1, ..., xn))` $ = 1 * x_1 * \dots * x_n$

In [4]:
val s = (x: Int, y: Int) => x + y             // (S)
val p = (x: Int, y: Int) => x * y             // (P)

val xs: List[Int] = (2 to 4).toList

s: (Int, Int) => Int = ammonite.$sess.cmd4$Helper$$Lambda$2369/1046834897@1a0fc462
p: (Int, Int) => Int = ammonite.$sess.cmd4$Helper$$Lambda$2370/1181697356@33f0438c
xs: List[Int] = List(2, 3, 4)

### FoldLeft and ReduceLeft

Notice that the `reduceLeft` method is defined in terms of `foldLeft`.


```scala
def reduceLeft(op: (A, A) => A): A = this match {
    case Nil     => error("Nil.reduceLeft")    // (1)
    case x :: xs => (xs foldLeft x)(op)        // (2)
}

def foldLeft[B](z: B)(op: (B, A) => B): B = this match {
    case Nil     => z                          // (3)
    case x :: xs => (xs foldLeft op(z, x))(op) // (4)
}
```


For instance, the sum of a sequence of numbers:

In [5]:
xs reduceLeft s

res5: Int = 9

```text
> List(2, 3, 4) reduceLeft s
> (List(3, 4) foldLeft (2)) s                 // (2)
> (List(4) foldLeft (2 s 3)) s                // (4)
> (List(4) foldLeft 5) s                      // (S)
> (List() foldLeft (5 s 4) s                  // (4)
> (List() foldLeft 9) s                       // (S)
> 9                                           // (3)
```

Another example, the product of a sequence of numbers:

In [6]:
xs reduceLeft p

res6: Int = 24

```text
> List(2, 3, 4) reduceLeft p
> (List(3, 4) foldLeft (2)) p                 // (2)
> (List(4) foldLeft (2 p 3)) p                // (4)
> (List(4) foldLeft 6) p                      // (P)
> (List() foldLeft (6 p 4)) p                 // (4)
> (List() foldLeft 24) p                      // (P)
> 24                                          // (3)
```

### FoldRight and ReduceRight

Applications of `foldLeft` and `reduceLeft` expand to
left-leaning trees. . They have duals `foldRight` and `reduceRight`, which produce
right-leaning trees

```scala
def reduceRight(op: (A, A) => A): A = this match {
    case Nil      => error("Nil.reduceRight")        // (1)
    case x :: Nil => x                               // (2)
    case x :: xs  => op(x, xs reduceRight op)        // (3)
}

def foldRight[B](z: B)(op: (A, B) => B): B = this match {
    case Nil     => z                                // (4)
    case x :: xs => op(x, (xs foldRight z)(op))      // (5) 
}
```

For instance, the sum of a sequence of numbers:

In [7]:
xs reduceRight s

res7: Int = 9

```text
> List(2, 3, 4) reduceRight s
> s(2, List(3, 4) reduceRight s)              // (3)
> s(2, s(3, List(4) reduceRight s))           // (3)
> s(2, s(3, 4))                               // (2)
> s(2, 7)                                     // (S)
> 9                                           // (S)
```

In [8]:
xs reduceRight p

res8: Int = 24

```text
> List(2, 3, 4) reduceRight p
> p(2, List(3, 4) reduceRight p)              // (3)
> p(2, p(3, List(4) reduceRight p))           // (3)
> p(2, p(3, 4))                               // (2)
> p(2, 12)                                    // (P)
> 24                                          // (P)
```

### Left vs. Right Folding

**In most cases, left folding and reduce are preferred over right folding, because right folding is recursive but not tail-recursive**, 
which means it consumes one stack frame for each element in the list.

In [1]:
List.range(1, 10)

res1: List[Int] = List(1, 2, 3, 4, 5, 6, 7, 8, 9)

### Flatten a list

In [5]:
import scala.language.postfixOps

val xxs = List(
    List.range(1, 5),
    List.range(6, 10),
    List.range(11, 15))

def flatten[A](xs: List[List[A]]): List[A] = {
    (xs foldRight (Nil: List[A])) {(x, ys) => x ::: ys}
}

flatten(xxs)

import scala.language.postfixOps
xxs: List[List[Int]] = List(
  List(1, 2, 3, 4),
  List(6, 7, 8, 9),
  List(11, 12, 13, 14)
)
defined function flatten
res5_3: List[Int] = List(1, 2, 3, 4, 6, 7, 8, 9, 11, 12, 13, 14)

## Nested Mappings

We can employ higher-order list processing functions to express many computations that are normally expressed as nested loops in imperative
languages.

As an example, consider the following problem: Given a positive integer $n$, find all
pairs of positive integers $i$ and $j$, where $1 \leq j \leq i < n$ such that $i + j$ is prime.

For instance, if $n = 7$ the pairs are:

| i | j | i + j |
|---|---|-------|
| 2 | 1 | 3     |
| 3 | 2 | 5     |
| 4 | 1 | 5     |
| 4 | 3 | 7     |
| 5 | 2 | 7     |
| 6 | 1 | 7     |
| 6 | 5 | 11    |



In a first step, one generates the sequence of all pairs $(i, j)$ of integers such that $1 \leq j < i < n$.

In [8]:
val n = 7

List.range(1, n)
    .map(i => List.range(1, i).map(j => (i, j)))

n: Int = 7
res8_1: List[List[(Int, Int)]] = List(
  List(),
  List((2, 1)),
  List((3, 1), (3, 2)),
  List((4, 1), (4, 2), (4, 3)),
  List((5, 1), (5, 2), (5, 3), (5, 4)),
  List((6, 1), (6, 2), (6, 3), (6, 4), (6, 5))
)

Secondly, combine all sublists (flatting) using foldRight with the concat operator `:::`,

In [9]:
List.range(1, n)
    .map(i => List.range(1, i).map(j => (i, j)))
    .foldRight(List[(Int, Int)]()){(x, xs) => x ::: xs}

res9: List[(Int, Int)] = List(
  (2, 1),
  (3, 1),
  (3, 2),
  (4, 1),
  (4, 2),
  (4, 3),
  (5, 1),
  (5, 2),
  (5, 3),
  (5, 4),
  (6, 1),
  (6, 2),
  (6, 3),
  (6, 4),
  (6, 5)
)

finally, we filter the pairs that their sum is prime.

In [3]:
def isPrime(n: Int): Boolean = {
    def nestedIsPrime(n: Int, i: Int): Boolean = { 
        if (n <= 2) n == 2
        else if (n % i == 0) false
        else if (n < i * i) true
        else nestedIsPrime(n, i + 1)
    }
    nestedIsPrime(n, 2)
}

val n = 7

List.range(1, n)
    .map(i => List.range(1, i).map(j => (i, j)))
    .foldRight(List[(Int, Int)]()){(x, xs) => x ::: xs}
    .filter(pair => isPrime(pair._1 + pair._2))

defined function isPrime
n: Int = 7
res3_2: List[(Int, Int)] = List(
  (2, 1),
  (3, 2),
  (4, 1),
  (4, 3),
  (5, 2),
  (6, 1),
  (6, 5)
)

## Flattening Maps

The combination of mapping and then concatenating sublists
resulting from the map is so common that we there is a special method for it in class.

```scala
def flatMap[B](f: A => List[B]): List[B] = this match {
    case Nil     => Nil
    case x :: xs => f(x) ::: (xs flatMap f)
}
```

With `flatMap`, the pairs-whose-sum-is-prime expression could have been written
more concisely as follows.

In [1]:
def isPrime(n: Int): Boolean = {
    def nestedIsPrime(n: Int, i: Int): Boolean = { 
        if (n <= 2) n == 2
        else if (n % i == 0) false
        else if (n < i * i) true
        else nestedIsPrime(n, i + 1)
    }
    nestedIsPrime(n, 2)
}

val n = 7

List.range(1, n)
    .flatMap(i => List.range(1, i).map(j => (i, j)))
    .filter(pair => isPrime(pair._1 + pair._2))


defined function isPrime
n: Int = 7
res1_2: List[(Int, Int)] = List(
  (2, 1),
  (3, 2),
  (4, 1),
  (4, 3),
  (5, 2),
  (6, 1),
  (6, 5)
)

<p style="float: left;"><a href="merge-sort.ipynb" target="_blank">Previous</a></p>
<p style="float: right;"><a href="for-comprehensions.ipynb" target="_blank">Next</a></p>
<p style="text-align:center;">Tour of Scala</p>
<div style="clear: both;"></div>